In [24]:
import numpy as np
import math

import matplotlib.pyplot as plt
import cartopy.crs as crs
import cartopy.feature as cfeature
import matplotlib.colors as colors
from matplotlib.cm import get_cmap
from matplotlib import ticker
import matplotlib.gridspec as gridspec

from cartopy import config
import cartopy.crs as ccrs
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from cartopy.util import add_cyclic_point

from IPython.display import Image

from tqdm import tqdm
import os 

g = 9.8

In [25]:
directory = '/n/home12/hongwei/HONGWEI/lagranto.era5/Simulation/run_2000_10yr'

filename01 = ['/traj_20000101_trace.1']

filename04 = ['/traj_20000401_trace.1']

filename07 = ['/traj_20000701_trace.1']

filename10 = ['/traj_20001001_trace.1']

In [26]:
N_head = 5 # first 5 lines are head lines, not include data
N_column = 4+2

Nx = 36
Ny = 21
Nz = 5
N_parcel = 3780

if Nx*Ny*Nz!=N_parcel: print('ERROR: parcel number is wrong!')
    
N_inject = 1 # how many injecting times



In [76]:
# for DT/DZ compare:

Criterion = 2.0


In [77]:
# loop for all traj files for month 01

file1 = open(directory+filename01[0], 'r')
Lines = file1.readlines()
Nt = int( (len(Lines)-4)/N_parcel - 1 ) # 2928

lifetime_01   = np.zeros((Nz,Ny,Nx, N_inject))
lifetime_N_01 = np.zeros((Nz,Ny,Nx, N_inject))
lifetime_S_01 = np.zeros((Nz,Ny,Nx, N_inject))

for i_inject in tqdm(range(N_inject)):
    
    # (1) read original data from traj files
    data = np.zeros((N_parcel, Nt, N_column))

    file1 = open(directory+filename01[i_inject], 'r')
#     print('Begin: '+filename01[i_inject])
    Lines = file1.readlines()

    count = 0
    # Strips the newline character
    for line in Lines:
        count += 1
    
        if count>=5:
            i = count-5
            i_parcel = math.floor( i / (Nt+1) ) # Nt time lines plue 1 empty line
            i_t = i%(Nt+1)
        
            if i_t!=0:
                a = line.split()

                if i_t==1:
                    if float(a[0])!=0.0: print('ERROR: first time is not 0 !!!')
                
                data[i_parcel,i_t-1,0] = float(a[0]) # t [h]
                data[i_parcel,i_t-1,1] = float(a[1]) # lon
                data[i_parcel,i_t-1,2] = float(a[2]) # lat
                data[i_parcel,i_t-1,3] = float(a[3]) # lev
                data[i_parcel,i_t-1,4] = float(a[4]) # RHO
                data[i_parcel,i_t-1,5] = float(a[5]) # DTDP
                
    # (2) re-shape data(N_parcel, Nt, N_column) to data2(Nz,Ny,Nx,Nt,N_column)
    data2 = np.zeros((Nz,Ny,Nx,Nt,N_column))

    for ix in range(Nx):
        for iy in range(Ny):
            for iz in range(Nz):
                i_parcel = ix*(Ny*Nz) + iy*Nz + iz
                data2[iz,iy,ix,:,:] = data[i_parcel,:,:]
                
                
    # (3) calculate injected tracer lifetime in the stratosphere
    P_cont = 200 # [hPa]
    lifetime   = np.zeros((Nz,Ny,Nx))
    lifetime_N = np.zeros((Nz,Ny,Nx))
    lifetime_S = np.zeros((Nz,Ny,Nx))

    for iy in range(Ny):
        for iz in range(Nz):
            for ix in range(Nx):
                Num = 0
                Num_N = 0
                Num_S = 0
                for it in range(Nt-4):
                    Num = Num+1
                    
                    if data2[iz,iy,ix,it,2]>=0: # lat>=0
                        Num_N = Num_N + 1
                    else:
                        Num_S = Num_S + 1
                    
                    
                    RHO_1 = data2[iz,iy,ix,it,4] # [K/Pa]
                    RHO_2 = data2[iz,iy,ix,it+1,4] # [K/Pa]
                    RHO_3 = data2[iz,iy,ix,it+2,4] # [K/Pa]
                    RHO_4 = data2[iz,iy,ix,it+3,4] # [K/Pa]
                    RHO_5 = data2[iz,iy,ix,it+4,4] # [K/Pa]
                    
                    DTDP_1 = data2[iz,iy,ix,it,5] # [K/Pa]
                    DTDP_2 = data2[iz,iy,ix,it+1,5] # [K/Pa]
                    DTDP_3 = data2[iz,iy,ix,it+2,5] # [K/Pa]
                    DTDP_4 = data2[iz,iy,ix,it+3,5] # [K/Pa]
                    DTDP_5 = data2[iz,iy,ix,it+4,5] # [K/Pa]
                    
                    DTDZ_1 = DTDP_1 *(-g*RHO_1) * 1000.0 # [K/km]
                    DTDZ_2 = DTDP_2 *(-g*RHO_2) * 1000.0
                    DTDZ_3 = DTDP_3 *(-g*RHO_3) * 1000.0
                    DTDZ_4 = DTDP_4 *(-g*RHO_4) * 1000.0
                    DTDZ_5 = DTDP_5 *(-g*RHO_5) * 1000.0
                    
                    
                    P_1 = data2[iz,iy,ix,it,3]
                    P_2 = data2[iz,iy,ix,it+1,3]
                    P_3 = data2[iz,iy,ix,it+2,3]
                    
                    if P_1>400.0: # directly identified as tropopause
                        lifetime[iz,iy,ix] = Num # [day]
                        lifetime_N[iz,iy,ix] = Num_N
                        lifetime_S[iz,iy,ix] = Num_S
                        break                     
                    elif P_1>70.0: # the particle is lower than 70 hPa level
#                         if DTDP_1>0.0 and DTDP_2>0.0 and DTDP_3>0.0 and DTDP_4>0.0 and DTDP_5>0.0: 
#                         if DTDP_1>0.0 and DTDP_2>0.0 and DTDP_3>0.0: # temp lapse rate
                        if DTDZ_1<Criterion and DTDZ_2<Criterion and DTDZ_3<Criterion: 
                            if P_1<P_2 and P_2<P_3: # keep dropping down
                                lifetime[iz,iy,ix] = Num # [day]
                                lifetime_N[iz,iy,ix] = Num_N
                                lifetime_S[iz,iy,ix] = Num_S
                                break 
                
                
    # (4) create 2-D LAT and LON
    LON = np.zeros((Ny,Nx))
    LAT = np.zeros((Ny,Nx))

    for iy in range(Ny):
        for ix in range(Nx):
            LON[iy,ix] = data2[0,iy,ix,0,1]
            LAT[iy,ix] = data2[0,iy,ix,0,2]

            
    # (5) Sort lat dimension from min to max [-20:20]
    index = np.argsort(LAT[:,0])

    LAT[:,:] = LAT[index,:]
    LON[:,:] = LON[index,:]
    lifetime[:,:,:]   = lifetime[:,index,:]
    lifetime_N[:,:,:] = lifetime_N[:,index,:]
    lifetime_S[:,:,:] = lifetime_S[:,index,:]
    data2[:,:,:,:,:]  = data2[:,index,:,:,:]
              
    # (6) Finally, save lifetime into  lifetime_01[Nz,Ny,Nx,N_inject]
    lifetime_01[:,:,:,i_inject] = lifetime[:,:,:]
    lifetime_N_01[:,:,:,i_inject] = lifetime_N[:,:,:]
    lifetime_S_01[:,:,:,i_inject] = lifetime_S[:,:,:]
    
    file1.close()

100%|██████████| 1/1 [01:16<00:00, 76.09s/it]


In [78]:
data2[0,0,0,1,:]

array([ 2.400e+01, -5.250e+00, -1.852e+01,  9.500e+01,  1.750e-01,
        2.000e-03])

In [79]:
# save lifetime_01[Nz,Ny,Nx,N_inject] to Lifetime01.txt

with open('Lifetime01.txt', 'w') as f:
    for i_inject in range(N_inject):
        for ix in range(Nx):
            for iy in range(Ny):
                for iz in range(Nz):
                    f.write(str(lifetime_01[iz,iy,ix,i_inject]))
                    f.write('\n')    
f.close()

with open('Lifetime_N_01.txt', 'w') as f:
    for i_inject in range(N_inject):
        for ix in range(Nx):
            for iy in range(Ny):
                for iz in range(Nz):
                    f.write(str(lifetime_N_01[iz,iy,ix,i_inject]))
                    f.write('\n')    
f.close()

with open('Lifetime_S_01.txt', 'w') as f:
    for i_inject in range(N_inject):
        for ix in range(Nx):
            for iy in range(Ny):
                for iz in range(Nz):
                    f.write(str(lifetime_S_01[iz,iy,ix,i_inject]))
                    f.write('\n')    
f.close()

In [80]:
# loop for all traj files for month 04

file1 = open(directory+filename04[0], 'r')
Lines = file1.readlines()
Nt = int( (len(Lines)-4)/N_parcel - 1 ) # 2928

lifetime_04 = np.zeros((Nz,Ny,Nx, N_inject))

for i_inject in tqdm(range(N_inject)):
    
    # (1) read original data from traj files
    data = np.zeros((N_parcel, Nt, N_column))

    file1 = open(directory+filename04[i_inject], 'r')
#     print('Begin: '+filename04[i_inject])
    Lines = file1.readlines()

    count = 0
    # Strips the newline character
    for line in Lines:
        count += 1
    
        if count>=5:
            i = count-5
            i_parcel = math.floor( i / (Nt+1) ) # Nt time lines plue 1 empty line
            i_t = i%(Nt+1)
        
            if i_t!=0:
                a = line.split()

                if i_t==1:
                    if float(a[0])!=0.0: print('ERROR: first time is not 0 !!!')
                
                data[i_parcel,i_t-1,0] = float(a[0]) # t [h]
                data[i_parcel,i_t-1,1] = float(a[1]) # lon
                data[i_parcel,i_t-1,2] = float(a[2]) # lat
                data[i_parcel,i_t-1,3] = float(a[3]) # lev
                data[i_parcel,i_t-1,4] = float(a[4]) # RHO
                data[i_parcel,i_t-1,5] = float(a[5]) # DTDP
                
                
    # (2) re-shape data(N_parcel, Nt, N_column) to data2(Nz,Ny,Nx,Nt,N_column)
    data2 = np.zeros((Nz,Ny,Nx,Nt,N_column))

    for ix in range(Nx):
        for iy in range(Ny):
            for iz in range(Nz):
                i_parcel = ix*(Ny*Nz) + iy*Nz + iz
                data2[iz,iy,ix,:,:] = data[i_parcel,:,:]
                
                
    # (3) calculate injected tracer lifetime in the stratosphere
    P_cont = 200 # [hPa]
    lifetime   = np.zeros((Nz,Ny,Nx))
    lifetime_N = np.zeros((Nz,Ny,Nx))
    lifetime_S = np.zeros((Nz,Ny,Nx))

    for iy in range(Ny):
        for iz in range(Nz):
            for ix in range(Nx):
                Num = 0
                Num_N = 0
                Num_S = 0
                for it in range(Nt-4):
                    Num = Num+1
                    
                    if data2[iz,iy,ix,it,2]>=0: # lat>=0
                        Num_N = Num_N + 1
                    else:
                        Num_S = Num_S + 1
                    

                    RHO_1 = data2[iz,iy,ix,it,4] # [K/Pa]
                    RHO_2 = data2[iz,iy,ix,it+1,4] # [K/Pa]
                    RHO_3 = data2[iz,iy,ix,it+2,4] # [K/Pa]
                    RHO_4 = data2[iz,iy,ix,it+3,4] # [K/Pa]
                    RHO_5 = data2[iz,iy,ix,it+4,4] # [K/Pa]
                    
                    DTDP_1 = data2[iz,iy,ix,it,5] # [K/Pa]
                    DTDP_2 = data2[iz,iy,ix,it+1,5] # [K/Pa]
                    DTDP_3 = data2[iz,iy,ix,it+2,5] # [K/Pa]
                    DTDP_4 = data2[iz,iy,ix,it+3,5] # [K/Pa]
                    DTDP_5 = data2[iz,iy,ix,it+4,5] # [K/Pa]
                    
                    DTDZ_1 = DTDP_1 *(-g*RHO_1) * 1000.0 # [K/km]
                    DTDZ_2 = DTDP_2 *(-g*RHO_2) * 1000.0
                    DTDZ_3 = DTDP_3 *(-g*RHO_3) * 1000.0
                    DTDZ_4 = DTDP_4 *(-g*RHO_4) * 1000.0
                    DTDZ_5 = DTDP_5 *(-g*RHO_5) * 1000.0
                    
                    
                    P_1 = data2[iz,iy,ix,it,3]
                    P_2 = data2[iz,iy,ix,it+1,3]
                    P_3 = data2[iz,iy,ix,it+2,3]
                    
                    if P_1>400.0: # directly identified as tropopause
                        lifetime[iz,iy,ix] = Num # [day]
                        lifetime_N[iz,iy,ix] = Num_N
                        lifetime_S[iz,iy,ix] = Num_S
                        break                     
                    elif P_1>70.0: # the particle is lower than 70 hPa level
#                         if DTDP_1>0.0 and DTDP_2>0.0 and DTDP_3>0.0 and DTDP_4>0.0 and DTDP_5>0.0: #
#                         if DTDP_1>0.0 and DTDP_2>0.0 and DTDP_3>0.0: # temp lapse rate
                        if DTDZ_1<Criterion and DTDZ_2<Criterion and DTDZ_3<Criterion: 
                            if P_1<P_2 and P_2<P_3: # keep dropping down
                                lifetime[iz,iy,ix] = Num # [day]
                                lifetime_N[iz,iy,ix] = Num_N
                                lifetime_S[iz,iy,ix] = Num_S
                                break               
                

    # (4) create 2-D LAT and LON
    LON = np.zeros((Ny,Nx))
    LAT = np.zeros((Ny,Nx))

    for iy in range(Ny):
        for ix in range(Nx):
            LON[iy,ix] = data2[0,iy,ix,0,1]
            LAT[iy,ix] = data2[0,iy,ix,0,2]

            
    # (5) Sort lat dimension from min to max [-20:20]
    index = np.argsort(LAT[:,0])

    LAT[:,:] = LAT[index,:]
    LON[:,:] = LON[index,:]
    lifetime[:,:,:] = lifetime[:,index,:]
    data2[:,:,:,:,:] = data2[:,index,:,:,:]
              
    # (6) Finally, save lifetime into  lifetime_04[Nz,Ny,Nx,N_inject]
    lifetime_04[:,:,:,i_inject] = lifetime[:,:,:]
        
    file1.close()

100%|██████████| 1/1 [01:15<00:00, 75.32s/it]


In [81]:
# save lifetime_04[Nz,Ny,Nx,N_inject] to Lifetime04.txt

with open('Lifetime04.txt', 'w') as f:
    for i_inject in range(N_inject):
        for ix in range(Nx):
            for iy in range(Ny):
                for iz in range(Nz):
                    f.write(str(lifetime_04[iz,iy,ix,i_inject]))
                    f.write('\n')    
f.close()

In [82]:
# loop for all traj files for month 07

file1 = open(directory+filename07[0], 'r')
Lines = file1.readlines()
Nt = int( (len(Lines)-4)/N_parcel - 1 ) # 2928

lifetime_07 = np.zeros((Nz,Ny,Nx, N_inject))

for i_inject in tqdm(range(N_inject)):
    
    # (1) read original data from traj files
    data = np.zeros((N_parcel, Nt, N_column))

    file1 = open(directory+filename07[i_inject], 'r')
#     print('Begin: '+filename07[i_inject])
    Lines = file1.readlines()

    count = 0
    # Strips the newline character
    for line in Lines:
        count += 1
    
        if count>=5:
            i = count-5
            i_parcel = math.floor( i / (Nt+1) ) # Nt time lines plue 1 empty line
            i_t = i%(Nt+1)
        
            if i_t!=0:
                a = line.split()

                if i_t==1:
                    if float(a[0])!=0.0: print('ERROR: first time is not 0 !!!')
                
                data[i_parcel,i_t-1,0] = float(a[0]) # t [h]
                data[i_parcel,i_t-1,1] = float(a[1]) # lon
                data[i_parcel,i_t-1,2] = float(a[2]) # lat
                data[i_parcel,i_t-1,3] = float(a[3]) # lev
                data[i_parcel,i_t-1,4] = float(a[4]) # RHO
                data[i_parcel,i_t-1,5] = float(a[5]) # DTDP
                
                
    # (2) re-shape data(N_parcel, Nt, N_column) to data2(Nz,Ny,Nx,Nt,N_column)
    data2 = np.zeros((Nz,Ny,Nx,Nt,N_column))

    for ix in range(Nx):
        for iy in range(Ny):
            for iz in range(Nz):
                i_parcel = ix*(Ny*Nz) + iy*Nz + iz
                data2[iz,iy,ix,:,:] = data[i_parcel,:,:]
                
                
    # (3) calculate injected tracer lifetime in the stratosphere
    P_cont = 200 # [hPa]
    lifetime   = np.zeros((Nz,Ny,Nx))
    lifetime_N = np.zeros((Nz,Ny,Nx))
    lifetime_S = np.zeros((Nz,Ny,Nx))

    for iy in range(Ny):
        for iz in range(Nz):
            for ix in range(Nx):
                Num = 0
                Num_N = 0
                Num_S = 0
                for it in range(Nt-4):
                    Num = Num+1
                    
                    if data2[iz,iy,ix,it,2]>=0: # lat>=0
                        Num_N = Num_N + 1
                    else:
                        Num_S = Num_S + 1
                    

                    RHO_1 = data2[iz,iy,ix,it,4] # [K/Pa]
                    RHO_2 = data2[iz,iy,ix,it+1,4] # [K/Pa]
                    RHO_3 = data2[iz,iy,ix,it+2,4] # [K/Pa]
                    RHO_4 = data2[iz,iy,ix,it+3,4] # [K/Pa]
                    RHO_5 = data2[iz,iy,ix,it+4,4] # [K/Pa]
                    
                    DTDP_1 = data2[iz,iy,ix,it,5] # [K/Pa]
                    DTDP_2 = data2[iz,iy,ix,it+1,5] # [K/Pa]
                    DTDP_3 = data2[iz,iy,ix,it+2,5] # [K/Pa]
                    DTDP_4 = data2[iz,iy,ix,it+3,5] # [K/Pa]
                    DTDP_5 = data2[iz,iy,ix,it+4,5] # [K/Pa]
                    
                    DTDZ_1 = DTDP_1 *(-g*RHO_1) * 1000.0 # [K/km]
                    DTDZ_2 = DTDP_2 *(-g*RHO_2) * 1000.0
                    DTDZ_3 = DTDP_3 *(-g*RHO_3) * 1000.0
                    DTDZ_4 = DTDP_4 *(-g*RHO_4) * 1000.0
                    DTDZ_5 = DTDP_5 *(-g*RHO_5) * 1000.0
                    
                    
                    P_1 = data2[iz,iy,ix,it,3]
                    P_2 = data2[iz,iy,ix,it+1,3]
                    P_3 = data2[iz,iy,ix,it+2,3]
                    
                    if P_1>400.0: # directly identified as tropopause
                        lifetime[iz,iy,ix] = Num # [day]
                        lifetime_N[iz,iy,ix] = Num_N
                        lifetime_S[iz,iy,ix] = Num_S
                        break                     
                    elif P_1>70.0: # the particle is lower than 70 hPa level
#                         if DTDP_1>0.0 and DTDP_2>0.0 and DTDP_3>0.0 and DTDP_4>0.0 and DTDP_5>0.0: 
#                         if DTDP_1>0.0 and DTDP_2>0.0 and DTDP_3>0.0: # temp lapse rate
                        if DTDZ_1<Criterion and DTDZ_2<Criterion and DTDZ_3<Criterion: 
                            if P_1<P_2 and P_2<P_3: # keep dropping down
                                lifetime[iz,iy,ix] = Num # [day]
                                lifetime_N[iz,iy,ix] = Num_N
                                lifetime_S[iz,iy,ix] = Num_S
                                break 
                
                
    # (4) create 2-D LAT and LON
    LON = np.zeros((Ny,Nx))
    LAT = np.zeros((Ny,Nx))

    for iy in range(Ny):
        for ix in range(Nx):
            LON[iy,ix] = data2[0,iy,ix,0,1]
            LAT[iy,ix] = data2[0,iy,ix,0,2]

            
    # (5) Sort lat dimension from min to max [-20:20]
    index = np.argsort(LAT[:,0])

    LAT[:,:] = LAT[index,:]
    LON[:,:] = LON[index,:]
    lifetime[:,:,:] = lifetime[:,index,:]
    data2[:,:,:,:,:] = data2[:,index,:,:,:]
              
    # (6) Finally, save lifetime into  lifetime_07[Nz,Ny,Nx,N_inject]
    lifetime_07[:,:,:,i_inject] = lifetime[:,:,:]
        
    file1.close()

100%|██████████| 1/1 [01:15<00:00, 75.78s/it]


In [83]:
# save lifetime_07[Nz,Ny,Nx,N_inject] to Lifetime07.txt

with open('Lifetime07.txt', 'w') as f:
    for i_inject in range(N_inject):
        for ix in range(Nx):
            for iy in range(Ny):
                for iz in range(Nz):
                    f.write(str(lifetime_07[iz,iy,ix,i_inject]))
                    f.write('\n')    
f.close()

In [84]:
# loop for all traj files for month 10

file1 = open(directory+filename10[0], 'r')
Lines = file1.readlines()
Nt = int( (len(Lines)-4)/N_parcel - 1 ) # 2928

lifetime_10 = np.zeros((Nz,Ny,Nx, N_inject))

for i_inject in tqdm(range(N_inject)):
    
    # (1) read original data from traj files
    data = np.zeros((N_parcel, Nt, N_column))

    file1 = open(directory+filename10[i_inject], 'r')
#     print('Begin: '+filename10[i_inject])
    Lines = file1.readlines()

    count = 0
    # Strips the newline character
    for line in Lines:
        count += 1
    
        if count>=5:
            i = count-5
            i_parcel = math.floor( i / (Nt+1) ) # Nt time lines plue 1 empty line
            i_t = i%(Nt+1)
        
            if i_t!=0:
                a = line.split()

                if i_t==1:
                    if float(a[0])!=0.0: print('ERROR: first time is not 0 !!!')
                
                data[i_parcel,i_t-1,0] = float(a[0]) # t [h]
                data[i_parcel,i_t-1,1] = float(a[1]) # lon
                data[i_parcel,i_t-1,2] = float(a[2]) # lat
                data[i_parcel,i_t-1,3] = float(a[3]) # lev
                data[i_parcel,i_t-1,4] = float(a[4]) # RHO
                data[i_parcel,i_t-1,5] = float(a[5]) # DTDP
                
                
    # (2) re-shape data(N_parcel, Nt, N_column) to data2(Nz,Ny,Nx,Nt,N_column)
    data2 = np.zeros((Nz,Ny,Nx,Nt,N_column))

    for ix in range(Nx):
        for iy in range(Ny):
            for iz in range(Nz):
                i_parcel = ix*(Ny*Nz) + iy*Nz + iz
                data2[iz,iy,ix,:,:] = data[i_parcel,:,:]
                
                
    # (3) calculate injected tracer lifetime in the stratosphere
    P_cont = 200 # [hPa]
    lifetime   = np.zeros((Nz,Ny,Nx))
    lifetime_N = np.zeros((Nz,Ny,Nx))
    lifetime_S = np.zeros((Nz,Ny,Nx))

    for iy in range(Ny):
        for iz in range(Nz):
            for ix in range(Nx):
                Num = 0
                Num_N = 0
                Num_S = 0
                for it in range(Nt-4):
                    Num = Num+1
                    
                    if data2[iz,iy,ix,it,2]>=0: # lat>=0
                        Num_N = Num_N + 1
                    else:
                        Num_S = Num_S + 1
                    

                    RHO_1 = data2[iz,iy,ix,it,4] # [K/Pa]
                    RHO_2 = data2[iz,iy,ix,it+1,4] # [K/Pa]
                    RHO_3 = data2[iz,iy,ix,it+2,4] # [K/Pa]
                    RHO_4 = data2[iz,iy,ix,it+3,4] # [K/Pa]
                    RHO_5 = data2[iz,iy,ix,it+4,4] # [K/Pa]
                    
                    DTDP_1 = data2[iz,iy,ix,it,5] # [K/Pa]
                    DTDP_2 = data2[iz,iy,ix,it+1,5] # [K/Pa]
                    DTDP_3 = data2[iz,iy,ix,it+2,5] # [K/Pa]
                    DTDP_4 = data2[iz,iy,ix,it+3,5] # [K/Pa]
                    DTDP_5 = data2[iz,iy,ix,it+4,5] # [K/Pa]
                    
                    DTDZ_1 = DTDP_1 *(-g*RHO_1) * 1000.0 # [K/km]
                    DTDZ_2 = DTDP_2 *(-g*RHO_2) * 1000.0
                    DTDZ_3 = DTDP_3 *(-g*RHO_3) * 1000.0
                    DTDZ_4 = DTDP_4 *(-g*RHO_4) * 1000.0
                    DTDZ_5 = DTDP_5 *(-g*RHO_5) * 1000.0
                    
                    
                    P_1 = data2[iz,iy,ix,it,3]
                    P_2 = data2[iz,iy,ix,it+1,3]
                    P_3 = data2[iz,iy,ix,it+2,3]
                    
                    if P_1>400.0: # directly identified as tropopause
                        lifetime[iz,iy,ix] = Num # [day]
                        lifetime_N[iz,iy,ix] = Num_N
                        lifetime_S[iz,iy,ix] = Num_S
                        break                     
                    elif P_1>70.0: # the particle is lower than 70 hPa level
#                         if DTDP_1>0.0 and DTDP_2>0.0 and DTDP_3>0.0 and DTDP_4>0.0 and DTDP_5>0.0: 
#                         if DTDP_1>0.0 and DTDP_2>0.0 and DTDP_3>0.0: # temp lapse rate
                        if DTDZ_1<Criterion and DTDZ_2<Criterion and DTDZ_3<Criterion: 
                            if P_1<P_2 and P_2<P_3: # keep dropping down
                                lifetime[iz,iy,ix] = Num # [day]
                                lifetime_N[iz,iy,ix] = Num_N
                                lifetime_S[iz,iy,ix] = Num_S
                                break 
                            
                
                
    # (4) create 2-D LAT and LON
    LON = np.zeros((Ny,Nx))
    LAT = np.zeros((Ny,Nx))

    for iy in range(Ny):
        for ix in range(Nx):
            LON[iy,ix] = data2[0,iy,ix,0,1]
            LAT[iy,ix] = data2[0,iy,ix,0,2]

            
    # (5) Sort lat dimension from min to max [-20:20]
    index = np.argsort(LAT[:,0])

    LAT[:,:] = LAT[index,:]
    LON[:,:] = LON[index,:]
    lifetime[:,:,:] = lifetime[:,index,:]
    data2[:,:,:,:,:] = data2[:,index,:,:,:]
              
    # (6) Finally, save lifetime into  lifetime_10[Nz,Ny,Nx,N_inject]
    lifetime_10[:,:,:,i_inject] = lifetime[:,:,:]
        
    file1.close()

100%|██████████| 1/1 [01:15<00:00, 75.49s/it]


In [85]:
# save lifetime_10[Nz,Ny,Nx,N_inject] to Lifetime10.txt

with open('Lifetime10.txt', 'w') as f:
    for i_inject in range(N_inject):
        for ix in range(Nx):
            for iy in range(Ny):
                for iz in range(Nz):
                    f.write(str(lifetime_10[iz,iy,ix,i_inject]))
                    f.write('\n')    
f.close()